In [1]:
# import packages
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
from pmdarima.arima import auto_arima

# for calculation
from math import sqrt
from sklearn.metrics import mean_squared_error

# to plot within notebook
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data80 = pd.read_csv('105v2.csv', parse_dates=True, index_col='DateTime',encoding="windows-1252") # или 105
data80.head(5)

,AffinityIndex,SerialNumber,H,Tmean,WmeanH,WindDirectionH(Â°),WminH,WmaxH,WmeanV,AtmosphericPressure,RelativeHumidity,TdewPoint(Â°C),WaterVaporElasticity,AbsolutusHumidity,DensityofAir,SpeedofSoundinAir
DateTime,,,,,,,,,,,,,,,,
2017-07-22 00:01:00,6,12427AMK-03,15,15.86,0.20,94.53,0.01,0.44,-0.13,750.45,100.0,15.86,17.99,14.53,1.1954,342.12
2017-07-22 00:02:00,6,12427AMK-03,15,15.88,0.28,104.28,0.15,0.53,-0.19,750.45,100.0,15.88,18.01,14.55,1.1953,342.14
2017-07-22 00:03:00,6,12427AMK-03,15,15.87,0.23,68.53,0.11,0.36,-0.19,750.45,100.0,15.87,17.99,14.54,1.1954,342.13
2017-07-22 00:04:00,6,12427AMK-03,15,15.95,0.17,47.04,0.01,0.33,-0.18,750.45,100.0,15.95,18.09,14.62,1.1950,342.18
2017-07-22 00:05:00,6,12427AMK-03,15,16.01,0.06,11.99,0.01,0.22,-0.19,750.45,100.0,16.01,18.16,14.68,1.1947,342.22


In [3]:
data80 = data80.groupby('DateTime')['Tmean'].sum().reset_index()

In [4]:
data80 = data80.set_index('DateTime')
data80.index

DatetimeIndex(['2017-07-22 00:01:00', '2017-07-22 00:02:00',
               '2017-07-22 00:03:00', '2017-07-22 00:04:00',
               '2017-07-22 00:05:00', '2017-07-22 00:06:00',
               '2017-07-22 00:07:00', '2017-07-22 00:08:00',
               '2017-07-22 00:09:00', '2017-07-22 00:10:00',
               ...
               '2020-04-09 23:50:00', '2020-04-09 23:51:00',
               '2020-04-09 23:52:00', '2020-04-09 23:53:00',
               '2020-04-09 23:54:00', '2020-04-09 23:55:00',
               '2020-04-09 23:56:00', '2020-04-09 23:57:00',
               '2020-04-09 23:58:00', '2020-04-09 23:59:00'],
              dtype='datetime64[ns]', name='DateTime', length=1183038, freq=None)

In [5]:
df = data80['Tmean'].resample('D').mean() # MS - месяц, D - день
df.interpolate(method='linear', inplace=True)

In [6]:
train_len = int(df.shape[0] * 0.8)
train_data, test_data = df[:train_len], df[train_len:]

y_train = train_data.values
y_test = test_data.values

print(f"{train_len} train samples")
print(f"{df.shape[0] - train_len} test samples")

794 train samples
199 test samples


In [7]:
from pmdarima.arima import ndiffs

kpss_diffs = ndiffs(y_train, alpha=0.05, test='kpss', max_d=6)
adf_diffs = ndiffs(y_train, alpha=0.05, test='adf', max_d=6)
n_diffs = max(adf_diffs, kpss_diffs)

print(f"Estimated differencing term: {n_diffs}")

Estimated differencing term: 1


In [8]:
auto = auto_arima(y_train, d=n_diffs, seasonal=False, stepwise=True,
                     suppress_warnings=True, error_action="ignore", max_p=6,
                     max_order=None, trace=True)

Performing stepwise search to minimize aic
Fit ARIMA(2,1,2)x(0,0,0,0) [intercept=True]; AIC=4135.592, BIC=4163.647, Time=0.362 seconds
Fit ARIMA(0,1,0)x(0,0,0,0) [intercept=True]; AIC=4212.680, BIC=4222.031, Time=0.023 seconds
Fit ARIMA(1,1,0)x(0,0,0,0) [intercept=True]; AIC=4210.043, BIC=4224.070, Time=0.095 seconds
Fit ARIMA(0,1,1)x(0,0,0,0) [intercept=True]; AIC=4204.953, BIC=4218.980, Time=0.094 seconds
Fit ARIMA(0,1,0)x(0,0,0,0) [intercept=False]; AIC=4210.686, BIC=4215.362, Time=0.015 seconds
Fit ARIMA(1,1,2)x(0,0,0,0) [intercept=True]; AIC=4135.777, BIC=4159.156, Time=0.262 seconds
Fit ARIMA(2,1,1)x(0,0,0,0) [intercept=True]; AIC=4148.403, BIC=4171.782, Time=0.235 seconds
Fit ARIMA(3,1,2)x(0,0,0,0) [intercept=True]; AIC=4136.324, BIC=4169.054, Time=0.463 seconds
Fit ARIMA(2,1,3)x(0,0,0,0) [intercept=True]; AIC=4136.139, BIC=4168.870, Time=0.715 seconds
Fit ARIMA(1,1,1)x(0,0,0,0) [intercept=True]; AIC=4183.661, BIC=4202.364, Time=0.229 seconds
Fit ARIMA(1,1,3)x(0,0,0,0) [intercep

In [9]:
print(auto.order)


(1, 1, 3)


In [10]:
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape

model = auto

def forecast_one_step():
    fc, conf_int = model.predict(n_periods=1, return_conf_int=True)
    return (
        fc.tolist()[0],
        np.asarray(conf_int).tolist()[0])

forecasts = []
confidence_intervals = []

for new_ob in y_test:
    fc, conf = forecast_one_step()
    forecasts.append(fc)
    confidence_intervals.append(conf)
    
    # Updates the existing model with a small number of MLE steps
    model.update(new_ob)
    
print(f"Mean squared error: {mean_squared_error(y_test, forecasts)}")
print(f"SMAPE: {smape(y_test, forecasts)}")

Mean squared error: 14.948492642137703
SMAPE: 54.45902576686635
